In [8]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode


In [9]:
import nltk
import spacy
nlp = spacy.load('en_core_web_sm')

In [10]:
class WordEncoder:
    def __init__(self):
        self.encoding_map = {}
        self.decoding_map = {}
        self.current_index = 43
    
    def encode_word(self, word):
        if word not in self.encoding_map:
            self.encoding_map[word] = self.current_index
            self.decoding_map[self.current_index] = word
            self.current_index += 1
        return self.encoding_map[word]
    
    def decode_word(self, encoded_word):
        if encoded_word in self.decoding_map:
            return self.decoding_map[encoded_word]
        else:
            raise ValueError("Invalid encoded word.")

In [11]:
encoder = WordEncoder()

In [12]:
df = pd.read_json('train.json')
df = df[df["category"]!='geometry']
df.head()

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,the banker ' s gain of a certain sum due 3 yea...,"""explanation : t = 3 years r = 10 % td = ( bg ...","a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d...",a,"divide(multiply(const_100, divide(multiply(36,...","multiply(n2,const_100)|multiply(n0,n1)|divide(...",gain
1,average age of students of an adult school is ...,"""explanation : let the original no . of studen...","a ) 1200 , b ) 120 , c ) 360 , d ) 240 , e ) n...",d,"multiply(divide(subtract(multiply(add(32, 4), ...","add(n2,n3)|multiply(n1,n2)|multiply(n1,#0)|sub...",general
2,sophia finished 2 / 3 of a book . she calculat...,let xx be the total number of pages in the boo...,"a ) 229 , b ) 270 , c ) 877 , d ) 266 , e ) 281",b,"divide(90, subtract(const_1, divide(2, 3)))","divide(n0,n1)|subtract(const_1,#0)|divide(n2,#1)",general
3,120 is what percent of 50 ?,"""50 * x = 120 - - > x = 2.4 - - > 2.4 expresse...","a ) 5 % , b ) 240 % , c ) 50 % , d ) 2 % , e )...",b,"multiply(divide(120, 50), const_100)","divide(n0,n1)|multiply(#0,const_100)|",gain
4,there are 10 girls and 20 boys in a classroom ...,"if girls is 10 and boys is 20 , then 10 / 20 ....","a ) 1 / 2 , b ) 1 / 3 , c ) 1 / 5 , d ) 10 / 3...",a,"divide(10, 20)","divide(n0,n1)",other


In [13]:
def cleanAnnotatedFormula(text) :
    text = text.replace(".","")
    tokens = re.findall(r'\b\w+\b', text)
    L = []
    for i in range(len(tokens)):
        if tokens[i].isdigit():
            L.append(int(tokens[i]))
    L = sorted(list(dict.fromkeys(L)))
    L = {i:"NUM{}".format(j) for i,j in zip(L,list(range(len(L))))}
    for i in range(len(tokens)):
        if tokens[i].isdigit():
            tokens[i] = L[int(tokens[i])]
    return tokens

In [14]:
df = df[[len(cleanAnnotatedFormula(i)) < 50 for i in df['annotated_formula'].values]]
df

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,the banker ' s gain of a certain sum due 3 yea...,"""explanation : t = 3 years r = 10 % td = ( bg ...","a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d...",a,"divide(multiply(const_100, divide(multiply(36,...","multiply(n2,const_100)|multiply(n0,n1)|divide(...",gain
1,average age of students of an adult school is ...,"""explanation : let the original no . of studen...","a ) 1200 , b ) 120 , c ) 360 , d ) 240 , e ) n...",d,"multiply(divide(subtract(multiply(add(32, 4), ...","add(n2,n3)|multiply(n1,n2)|multiply(n1,#0)|sub...",general
2,sophia finished 2 / 3 of a book . she calculat...,let xx be the total number of pages in the boo...,"a ) 229 , b ) 270 , c ) 877 , d ) 266 , e ) 281",b,"divide(90, subtract(const_1, divide(2, 3)))","divide(n0,n1)|subtract(const_1,#0)|divide(n2,#1)",general
3,120 is what percent of 50 ?,"""50 * x = 120 - - > x = 2.4 - - > 2.4 expresse...","a ) 5 % , b ) 240 % , c ) 50 % , d ) 2 % , e )...",b,"multiply(divide(120, 50), const_100)","divide(n0,n1)|multiply(#0,const_100)|",gain
4,there are 10 girls and 20 boys in a classroom ...,"if girls is 10 and boys is 20 , then 10 / 20 ....","a ) 1 / 2 , b ) 1 / 3 , c ) 1 / 5 , d ) 10 / 3...",a,"divide(10, 20)","divide(n0,n1)",other
...,...,...,...,...,...,...,...
29832,a man invests some money partly in 10 % stock ...,"""solution for an income of rs . 1 in 10 % stoc...","a ) 1 : 2 , b ) 3 : 5 , c ) 4 : 5 , d ) 16 : 1...",a,"divide(multiply(90, const_2), multiply(90, con...","multiply(n1,const_2)|multiply(n3,const_3)|divi...",other
29833,"average of 10 matches is 32 , how many runs on...","""explanation : average after 11 innings should...","a ) a ) 70 , b ) b ) 76 , c ) c ) 78 , d ) d )...",d,"subtract(multiply(add(32, 6), add(10, const_1)...","add(n1,n2)|add(n0,const_1)|multiply(n0,n1)|mul...",general
29834,solve below question 2 x + 1 = - 23,1 . subtract 1 from both sides : 2 x + 1 - 1 =...,"a ) - 8 , b ) - 9 , c ) - 12 , d ) - 4 , e ) 12",c,"divide(negate(add(23, 1)), 2)","add(n1,n2)|negate(#0)|divide(#1,n0)",general
29835,calculate 85184 ÷ ? = 352,"""answer let 85184 ÷ x = 242 then x = 85184 / 2...","a ) 241 , b ) 242 , c ) 244 , d ) 247 , e ) 240",b,"multiply(85184, 352)","multiply(n0,n1)|",general


In [15]:
df.category.unique()

array(['gain', 'general', 'other', 'physics', 'probability'], dtype=object)

In [16]:
level1DataFrame = df[['Problem','category']]
level1DataFrame

,Problem,category
0,the banker ' s gain of a certain sum due 3 yea...,gain
1,average age of students of an adult school is ...,general
2,sophia finished 2 / 3 of a book . she calculat...,general
3,120 is what percent of 50 ?,gain
4,there are 10 girls and 20 boys in a classroom ...,other
...,...,...
29832,a man invests some money partly in 10 % stock ...,other
29833,"average of 10 matches is 32 , how many runs on...",general
29834,solve below question 2 x + 1 = - 23,general
29835,calculate 85184 ÷ ? = 352,general


In [17]:
def firstLevelPreProcessingX(text):
    text = text.replace(".", " ")
    text = text.replace(",", "")
    doc = nlp(text.lower())
    tagged_words = np.array(list(map(lambda x: (x.text, x.pos_), doc)))

    nouns_raw = tagged_words[tagged_words[:, 1] == 'NOUN'][:, 0]
    encoded_nouns = [encoder.encode_word(i) for i in nouns_raw]
    if len(encoded_nouns) < 15:
            encoded_nouns = np.pad(encoded_nouns, (0, 15 - len(encoded_nouns)))

    verbs_raw = tagged_words[tagged_words[:, 1] == 'VERB'][:, 0]
    encoded_verbs = [encoder.encode_word(i) for i in verbs_raw]
    if len(encoded_verbs) < 10:
            encoded_verbs = np.pad(encoded_verbs, (0, 10 - len(encoded_verbs)))

    return np.array(list(encoded_nouns[:15]) +  list(encoded_verbs[:10]))


In [18]:
level1_X_raw = list(level1DataFrame['Problem'].values)
level1_Y_raw = list(level1DataFrame['category'].values)

In [19]:
level1_X = [firstLevelPreProcessingX(i) for i in level1_X_raw]

In [20]:
level1_Y = [encoder.encode_word(i) for i in level1_Y_raw]

In [21]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [22]:
model.fit(level1_X,level1_Y)

RandomForestClassifier()

In [23]:
df_test = pd.read_json("test.json")
df_test = df_test[df_test["category"]!='geometry']
level1_X_test = [firstLevelPreProcessingX(i) for i in list(df_test['Problem'].values)]
level1_Y_test = [encoder.encode_word(i) for i in list(df_test['category'].values)]

In [24]:
(list(level1_Y_test - model.predict(level1_X_test)).count(0)/len(level1_Y_test))*100

95.72710951526032

In [25]:
level2_df_general = df[["Problem","annotated_formula"]][df.category=="general"]
level2_df_gain = df[["Problem","annotated_formula"]][df.category=="gain"]
level2_df_other = df[["Problem","annotated_formula"]][df.category=="other"]
level2_df_physics = df[["Problem","annotated_formula"]][df.category=="physics"]
level2_df_probability = df[["Problem","annotated_formula"]][df.category=="probability"]

In [26]:
level2_df_general

,Problem,annotated_formula
1,average age of students of an adult school is ...,"multiply(divide(subtract(multiply(add(32, 4), ..."
2,sophia finished 2 / 3 of a book . she calculat...,"divide(90, subtract(const_1, divide(2, 3)))"
9,a student was asked to find 4 / 5 of a number ...,"divide(divide(multiply(multiply(36, divide(4, ..."
10,the average weight of 8 person ' s increases b...,"add(multiply(8, 1.5), 75)"
12,the average of 15 result is 60 . average of th...,"subtract(add(multiply(10, 10), multiply(10, 80..."
...,...,...
29830,if 16 percent of the students at a certain sch...,"divide(16, divide(subtract(100, 75), 100))"
29833,"average of 10 matches is 32 , how many runs on...","subtract(multiply(add(32, 6), add(10, const_1)..."
29834,solve below question 2 x + 1 = - 23,"divide(negate(add(23, 1)), 2)"
29835,calculate 85184 ÷ ? = 352,"multiply(85184, 352)"


In [27]:
def level2PreProcessingX(text) :
    text = text.replace(".", "")
    text = text.replace(",", "")
    doc = nlp(text.lower())
    tagged_words = np.array(list(map(lambda x: (x.text, x.pos_), doc)))
    verbs = []
    advs = []
    adjs = []
    nouns = []
    nums = []
    for i in tagged_words :
        if i[1]== "VERB" :
            verbs.append(i[0])
        if i[1]== "ADV" :
            advs.append(i[0])
        if i[1]== "ADJ" :
            adjs.append(i[0])
        if i[1]== "NOUN":
            nouns.append(i[0])
        if i[1]== "NUM" and i[0].isdigit():
            nums.append(int(unidecode(i[0])))
    verbs = list(dict.fromkeys(verbs))
    advs = list(dict.fromkeys(advs))
    adjs = list(dict.fromkeys(adjs))
    nums = list(dict.fromkeys(nums))
    L = {i:"NUM{}".format(j) for i,j in zip(sorted(nums),list(range(len(nums))))}
    nums = [L[i] for i in nums]

    encoded_verbs = [encoder.encode_word(i) for i in verbs]
    if len(encoded_verbs) < 5:
            encoded_verbs = np.pad(encoded_verbs, (0, 5 - len(encoded_verbs)))

    encoded_nums = [encoder.encode_word(i) for i in nums]
    if len(encoded_nums) < 10:
            encoded_nums = np.pad(encoded_nums, (0, 10 - len(encoded_nums)))

    encoded_advs = [encoder.encode_word(i) for i in advs]
    if len(encoded_advs) < 5:
            encoded_advs = np.pad(encoded_advs, (0, 5 - len(encoded_advs)))

    encoded_adjs = [encoder.encode_word(i) for i in adjs]
    if len(encoded_adjs) < 5:
            encoded_adjs = np.pad(encoded_adjs, (0, 5 - len(encoded_adjs)))

    encoded_nouns = [encoder.encode_word(i) for i in nouns]
    if len(encoded_nouns) < 15:
            encoded_nouns = np.pad(encoded_nouns, (0, 15 - len(encoded_nouns)))
    # print(len(encoded_verbs),len(encoded_nums),len(encoded_advs),len(encoded_adjs),len(encoded_nouns))
    return np.array(list(encoded_verbs[:5]) + list(encoded_nums[:10]) +  list(encoded_advs[:5]) + list(encoded_adjs[:5])  + list(encoded_nouns[:15]))
    

In [28]:
def level2PreProcessingY(text) :
    formula = cleanAnnotatedFormula(text)
    encoded_formula = [encoder.encode_word(i) for i in formula]
    if len(encoded_formula) < 50:
            encoded_formula = np.pad(encoded_formula, (0, 50 - len(encoded_formula)))
    return np.array(list(encoded_formula[:50]))

In [29]:
level2PreProcessingX(level2_df_general['Problem'].values[3])

array([ 105.,  106.,    0.,    0.,    0., 6092., 6093., 6094.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,  117., 6095.,    0.,    0.,    0.,  100.,  101.,
        102.,  103.,  101.,  104.,  103.,  100.,  101.,    0.,    0.,
          0.,    0.,    0.,    0.])

In [30]:
level2PreProcessingY(level2_df_general['annotated_formula'].values[3])

array([2014, 1969, 6092, 6093, 6094,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [31]:
level2generalX = [level2PreProcessingX(i) for i in level2_df_general["Problem"].values]

In [32]:
level2generalY = [level2PreProcessingY(i) for i in level2_df_general["annotated_formula"].values]

In [33]:
level2generaldataframeX = pd.DataFrame(level2generalX)
level2generaldataframeY = pd.DataFrame(level2generalY)


In [34]:
level2generaldataframeX

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,55.0,56.0,57.0,58.0,0.0,6094.0,6096.0,6093.0,6092.0,0.0,...,50.0,49.0,46.0,52.0,53.0,49.0,46.0,54.0,50.0,52.0
1,61.0,62.0,63.0,64.0,0.0,6092.0,6093.0,6094.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,97.0,57.0,98.0,99.0,0.0,6092.0,6093.0,6094.0,0.0,0.0,...,96.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,105.0,106.0,0.0,0.0,0.0,6092.0,6093.0,6094.0,0.0,0.0,...,104.0,103.0,100.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0
4,57.0,0.0,0.0,0.0,0.0,6094.0,6096.0,6093.0,6102.0,6092.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13193,1089.0,317.0,222.0,0.0,0.0,6092.0,6094.0,6093.0,0.0,0.0,...,65.0,50.0,1558.0,489.0,165.0,50.0,52.0,1558.0,489.0,0.0
13194,502.0,1019.0,149.0,0.0,0.0,6093.0,6094.0,6092.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13195,518.0,0.0,0.0,0.0,0.0,6093.0,6092.0,6094.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13196,340.0,0.0,0.0,0.0,0.0,6093.0,6092.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
level2generaldataframeY

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1969,1422,4364,1969,2014,6093,6092,6096,1969,6096,...,0,0,0,0,0,0,0,0,0,0
1,1422,6094,4364,6695,1422,6092,6093,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1422,1422,1969,1969,6094,1422,6092,6093,1422,6092,...,0,0,0,0,0,0,0,0,0,0
3,2014,1969,6092,6093,6094,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4364,2014,1969,6092,6092,1969,6092,6096,1969,6093,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13193,1422,6092,1422,4364,6094,6093,6094,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13194,4364,1969,2014,6094,6092,2014,6093,6695,1969,6093,...,0,0,0,0,0,0,0,0,0,0
13195,1422,6704,2014,6094,6092,6093,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13196,1969,6093,6092,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
from sklearn.tree import DecisionTreeClassifier


In [89]:
level2generalmodel = RandomForestClassifier()

level2generalmodel.fit(level2generaldataframeX,level2generaldataframeY)

MemoryError: could not allocate 275251200 bytes

In [70]:
t = 20
question = df_test[df_test['category']=='general']['Problem'].values[t]
equation = df_test[df_test['category']=='general']['annotated_formula'].values[t]
print(question)
print(equation)

find large number from below question the difference of two numbers is 1365 . on dividing the larger number by the smaller , we get 6 as quotient and the 15 as remainder
multiply(divide(subtract(1365, 15), subtract(6, const_1)), 6)


In [80]:
question = "Sarthak have 5 apples Thushar took 3 from him how many does he have now?"

In [81]:
print(level2PreProcessingX(question))
print(level2PreProcessingY(equation))

[ 526  317    0    0    0 6093 6092    0    0    0    0    0    0    0
    0 6139    0    0    0    0 6128    0    0    0    0 6725  861    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[1969 1422 4364 6094 6093 4364 6092 6695 6092    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [86]:
pred= level2generalmodel.predict([level2PreProcessingX(question)])
pred

d:\Softwares\Python\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:879: RuntimeWarning: invalid value encountered in divide
  proba /= len(self.estimators_)


array([[440, 440,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [88]:
L = []
for i in pred[0] :
    if i != 0  :
        L.append(encoder.decode_word(i))
    else :
        break
print(L)

['lcm', 'lcm']


In [42]:
['divide', 'add', 'multiply', '8', '8', '12', '16']


['divide', 'add', 'multiply', '8', '8', '12', '16']

In [60]:
print(cleanAnnotatedFormula(equation))

['multiply', 'NUM0', 'divide', 'NUM2', 'subtract', 'NUM1', 'NUM0']
